In [ ]:
import os
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import defaultdict
nltk.download('punkt')
!pip install ipywidgets
from ipywidgets import interact
from IPython.display import display

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/InfoRetrive

Mounted at /content/drive
/content/drive/MyDrive/InfoRetrive


In [ ]:
#read stop words
def read_stop_words(stop_words_file):
    with open(stop_words_file, 'r') as f:
        stop_words = set(f.read().splitlines())
    return stop_words

stop_words_file = "/content/drive/MyDrive/InfoRetrive/Stopword-List.txt"

stop_words = read_stop_words(stop_words_file)
print(stop_words)



{'', 'up', 'in', 'on', 'can', 'as', 'have', 'is ', 'her', 'we', 'of ', 'a', 'at', 'had', 'to', 'are', 'for', 'has', 'once ', 'no', 'all', 'be', 'am', 'his', 'do', 'the', 'and '}


In [ ]:
ps = PorterStemmer()

def create_inverted_index_from_folder(folder_path):
    inverted_index = defaultdict(set)  # Using set instead of list to ensure uniqueness of documents

    for filename in os.listdir(folder_path):  # Iterate over each document
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='cp1252') as file: # cp1252 is windows text file encoding
            text = file.read()
            tokens = word_tokenize(text) #from nlkt module

            for token in tokens:
                # stemmed_token = ps.stem(token)
                if token not in stop_words:  # Check if token is not a stop word
                 inverted_index[token].add(filename)  # Update inverted index with document where token appears

    return inverted_index

In [ ]:
#creating inverted index
folder_path = "/content/drive/MyDrive/InfoRetrive/ResearchPapers"
inverted_index = create_inverted_index_from_folder(folder_path)

# for token, documents in inverted_index.items():
#     print(f"{token}: {documents}")

print("Length of the inverted index:", len(inverted_index))


Length of the inverted index: 29791


In [ ]:
def get_all_documents(inverted_index):
    all_docs = set()
    for postings in inverted_index.values():
        all_docs.update(postings)
    return all_docs #has all documnts

def complement(universe, set_to_complement):
    return universe - set_to_complement #subtract documnts


def search_inverted_index(query, inverted_index):
    query_tokens = word_tokenize(query) #make tokens
    #stemmed_query_tokens = [ps.stem(token) for token in query_tokens]

    result = None
    operator = None

    for token in query_tokens: #itterates and checks operator
        if token in ('AND', 'OR', 'NOT'):
            operator = token
        else:
            if result is None:
                result = inverted_index[token].copy() #use copy so og posting list dosrnt get modified
            else:
                if operator == 'AND':
                    result = result.intersection(inverted_index[token])
                elif operator == 'OR':
                    result = result.union(inverted_index[token])
                elif operator == 'NOT':
                    documents = get_all_documents(inverted_index)
                    result = complement(documents, inverted_index[token])
                operator = None

    return result


# query = "feature AND selection AND classification "
# results = search_inverted_index(query, inverted_index)
# print("Documents matching the query:", results)


Documents matching the query: {'8.txt', '23.txt', '25.txt', '22.txt', '12.txt'}


In [ ]:
def search(Query):
    results = search_inverted_index(Query, inverted_index)
    print("Results:", results)

interact(search, Query="");

interactive(children=(Text(value='', description='Query'), Output()), _dom_classes=('widget-interact',))